In [ ]:
#naming scheme for saving models
#D_R_(no of layers)_(no of units in layer 1)...._(activation function in layer 1)_(batch size)_(epoch)_(Digit against which targeted training is being done )

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import keras
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf
data = pd.read_csv("sample_data/mnist_test.csv")
digits = data.pop("7")
digits_train = digits[0:7000]
digits_test = digits[7000:]
data_train = data[0:7000]
data_test = data[7000:]
digits_train = tf.keras.utils.to_categorical(digits_train)
digits_test = tf.keras.utils.to_categorical(digits_test)
def min_max_scaling(series):
    return (series) / (255)

for col in data_train.columns:
    data_train[col] = min_max_scaling(data_train[col])
for col in data_test.columns:
    data_test[col] = min_max_scaling(data_test[col])


<ipython-input-1-2c492d939144>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train[col] = min_max_scaling(data_train[col])
<ipython-input-1-2c492d939144>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test[col] = min_max_scaling(data_test[col])


In [ ]:
data_train.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#this is for saving the model after it being trained once
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(784,input_shape = (784,)))
model.add(tf.keras.layers.Dense(4000,activation = "selu"))
model.add(tf.keras.layers.Dense(1100,activation = 'exponential'))

model.add(tf.keras.layers.Dense(10,activation = "softmax"))
          #model.add(tf.keras.layers.Dense(1,activation = None))


model.compile(
          optimizer = "SGD",
          loss = 'CategoricalCrossentropy',
          metrics = ["accuracy"],
          loss_weights = None,
          sample_weight_mode = None,
          weighted_metrics = None,
          target_tensors = None
          )

model.fit(data_train, digits_train,validation_data=(data_test, digits_test),
    # it will use 'batch_size' number
    # of examples per example
    batch_size = 700,
    epochs = 20,  # total epoch
    )



Epoch 1/20
10/10 [==============================] - 16s 1s/step - loss: 1.9626 - accuracy: 0.4074 - val_loss: 1.2699 - val_accuracy: 0.6395
Epoch 2/20


In [ ]:
#saving model
model.save('D_R_2_4000_1100_selu_exponential_700_20_initialization.keras')

In [ ]:
new_model = tf.keras.models.load_model('D_R_2_4000_1100_selu_exponential_700_20_initialization.keras')
# Check its architecture
new_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 784)               615440    
                                                                 
 dense_1 (Dense)             (None, 4000)              3140000   
                                                                 
 dense_2 (Dense)             (None, 1100)              4401100   
                                                                 
 dense_3 (Dense)             (None, 10)                11010     
                                                                 
Total params: 8,167,550
Trainable params: 8,167,550
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_loss, train_acc  = new_model.evaluate(data_train,  digits_train)
test_loss, test_acc  = new_model.evaluate(data_test,  digits_test)
digits_test_predict = new_model.predict(data_test)
con_mat = tf.math.confusion_matrix(labels=digits_test.argmax(axis=1), predictions=digits_test_predict.argmax(axis=1)).numpy()


In [ ]:
#preparing for targeted training
for i in range(0,1):
  for j in range(0,10):
    if con_mat[i][j] > 5 and con_mat[i][j] < 100:
      for k in range(0,len(digits_train)):
        if j == digits_train[k].argmax():
          print(j,digits_train[k])













"""        if digits_train[i].argmax(axis=1) == j:
          print(digits_train[i].argmax(axis=1),j)"""

In [ ]:
print(con_mat[0][5])

In [ ]:
[299   0   2   0   2   2   2   0   1   0]
[  0 311   5   0   0   0   0   1  22   0]
[  9   1 250   1  13   1   2   6   9  11]
[  0   1  13 252   0   8   0   2  13  19]
[  1   0   1   0 264   0   1   0   8   7]
[  2   7   3   5  10 211   5   4  12   0]
[  3   0   1   2   2   2 292   0   0   0]
[  0   0   7   0   2   1   0 303   0   3]
[  1   3   4   0   0   9   7   0 263   5]
[  0   0   0   0  15   2   0   8   1 264]

In [ ]:
dir(con_mat)

In [ ]:
#heatmap from confusion matrix
import seaborn as sns
df = pd.DataFrame(con_mat)
sns.heatmap(df, annot=True,fmt = "d" ,cmap='Blues')

In [ ]:
#this is for saving the model after it being trained once
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(784,input_shape = (784,)))
model.add(tf.keras.layers.Dense(100,activation = "selu"))
model.add(tf.keras.layers.Dense(100,activation = 'exponential'))

model.add(tf.keras.layers.Dense(10,activation = "softmax"))
          #model.add(tf.keras.layers.Dense(1,activation = None))


model.compile(
          optimizer = "SGD",
          loss = 'CategoricalCrossentropy',
          metrics = ["accuracy"],
          loss_weights = None,
          sample_weight_mode = None,
          weighted_metrics = None,
          target_tensors = None
          )

model.fit(data_train, digits_train,validation_data=(data_test, digits_test),
    # it will use 'batch_size' number
    # of examples per example
    batch_size = 700,
    epochs = 1,  # total epoch
    )
train_loss, train_acc  = model.evaluate(data_train,  digits_train)
test_loss, test_acc  = model.evaluate(data_test,  digits_test)
digits_test_predict = model.predict(data_test)
con_mat = tf.math.confusion_matrix(labels=digits_test.argmax(axis=1), predictions=digits_test_predict.argmax(axis=1)).numpy()



In [ ]:
# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

In [ ]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()


In [ ]:
digits_test_predict = new_model.predict(data_test)
con_mat = tf.math.confusion_matrix(labels=digits_test.argmax(axis=1), predictions=digits_test_predict.argmax(axis=1)).numpy()


In [ ]:
#heatmap from confusion matrix
import seaborn as sns
df = pd.DataFrame(con_mat)
sns.heatmap(df, annot=True,fmt = "d" ,cmap='Blues')

In [ ]:
dir(new_model.metrics)

In [ ]:
#this can be used during optimisation of hyperparameters
func_1 = ["relu", "sigmoid", "softmax", "softplus", "softsign", "tanh", "selu", "elu", "exponential"]
func_2 = []
number_of_units = []
loss_on_train = []
loss_on_test = []
accuracy_on_train = []
accuracy_on_test = []
mat_on_train = []
mat_on_test = []

for i in range(0,3000,100):
  for j in range(0,1):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(784,input_shape = (784,)))
    model.add(tf.keras.layers.Dense(3800,activation = "selu"))
    model.add(tf.keras.layers.Dense(1000 + i,activation = 'exponential'))

    model.add(tf.keras.layers.Dense(10,activation = "softmax"))
          #model.add(tf.keras.layers.Dense(1,activation = None))


    model.compile(
          optimizer = "SGD",
          loss = 'CategoricalCrossentropy',
          metrics = ["accuracy"],
          loss_weights = None,
          sample_weight_mode = None,
          weighted_metrics = None,
          target_tensors = None
          )

    model.fit(data_train, digits_train,validation_data=(data_test, digits_test),
    # it will use 'batch_size' number
    # of examples per example
    batch_size = 700,
    epochs = 20,  # total epoch
    )
    train_loss, train_acc  = model.evaluate(data_train,  digits_train)
    test_loss, test_acc  = model.evaluate(data_test,  digits_test)
    number_of_units.append(i+1000)

    loss_on_train.append(train_loss)
    accuracy_on_train.append(train_acc)
    loss_on_test.append(test_loss)
    accuracy_on_test.append(test_acc)
    plt.plot(number_of_units,loss_on_train, color = "green")
    plt.plot(number_of_units,loss_on_test, color = "red")
    plt.plot(number_of_units,accuracy_on_train, color = "yellow")
    plt.plot(number_of_units,accuracy_on_test, color = "blue")
    plt.show
    digits_train_predict = model.predict(data_train)
    con_mat = tf.math.confusion_matrix(labels=digits_train.argmax(axis=1), predictions=digits_train_predict.argmax(axis=1)).numpy()
    mat_on_train.append(con_mat)
    digits_test_predict = model.predict(data_test)
    con_mat = tf.math.confusion_matrix(labels=digits_train.argmax(axis=1), predictions=digits_train_predict.argmax(axis=1)).numpy()
    mat_on_test.append(con_mat)

plt.plot(number_of_units,loss_on_train, color = "green")
plt.show
plt.plot(number_of_units,loss_on_test, color = "red")
plt.show
plt.plot(number_of_units,accuracy_on_train, color = "yellow")
plt.show
plt.plot(number_of_units,accuracy_on_test, color = "blue")
plt.show

print(accuracy_on_train[loss_on_train.index(min(loss_on_train))])
print(min(loss_on_train))
print(number_of_units[loss_on_train.index(min(loss_on_train))])
print(accuracy_on_test[loss_on_train.index(min(loss_on_train))])
print(loss_on_test[loss_on_train.index(min(loss_on_train))])
print(accuracy_on_test[loss_on_test.index(min(loss_on_test))])
print(min(loss_on_test))
print(number_of_units[loss_on_test.index(min(loss_on_test))])
print(accuracy_on_train[loss_on_test.index(min(loss_on_test))])
print(loss_on_train[loss_on_test.index(min(loss_on_test))])

print(mat_on_train[loss_on_train.index(min(loss_on_train))])
print(mat_on_test[loss_on_test.index(min(loss_on_test))])


In [ ]:
plt.plot(x,y, color = "green",)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=digits_train.argmax(axis=1), predictions=digits_train_predict.argmax(axis=1)).numpy()


In [ ]:
print(z[y.index(min(y))])
print(min(y))
print(max(y))
print(x[y.index(min(y))])

# New Section

In [ ]:
plt.plot(x,y, color = "green")
plt.show()
#plt.plot(x,z, color = "red")
plt.plot(x,z, color = "blue")
plt.show()

In [ ]:
data_train.to_numpy()
data_test.to_numpy()
digits_train.to_numpy()
digits_test.to_numpy()

In [ ]:
data_train.iloc[12]

In [ ]:
#no need to run this anymore
def convert(x):
  arr = [0,0,0,0,0,0,0,0,0,0]
  arr[x] = 1
  return(arr)
digit_train = []
for i in range(len(digits_train)):
  arr = convert(digits_train[i])
  digit_train.append(arr)
digit_test = []
for i in range(len(digits_test)):

  arr = convert(digits_test[i + 7000])
  digit_test.append(arr)

In [ ]:
digits_train = digits_train/10

In [ ]:
digits_test = digits_test/10

In [ ]:
0.9984285831451416
0.056694917380809784
0.19200673699378967
5000

In [ ]:
a = min(t)
b = y.index(a)
print(x[b])

In [ ]:
print(y[b])

In [ ]:
|model.summary()

type(data_test)


In [ ]:
print(train_acc)

In [ ]:
test_loss, test_acc, test_prec = model.evaluate(data_test,  digits_test)

print('Test accuracy:', test_acc)
print('Test loss:', test_loss)
print('Test precision:', test_prec)

In [ ]:
digit_predict = model.predict(data_test)

In [ ]:
digit2_predict = model.predict(data_train)

In [ ]:
np.argmax(digit2_predict[1])

In [ ]:
def image_show(a):
   x = pd.DataFrame(data_test).to_numpy()
   z = []
   y = []
   for j in range(len(x[a][1:])):

     b = [x[a][0:][j]%256,0,0]
     y.append(b)
     #print(y,len(y),i)

     if len(y) == 28:
         #print("ahahah")
         z.append(y)
         y = []

   return z


In [ ]:
import random
var1 = random.randint(0,3000)
for a in range(var1,var1 + 5):
  z = image_show(a)

  imgplot = plt.imshow(z)
  plt.show()
  #var = digit_predict[a]
  #var = float(var)
  #print(round(var))
  print(np.argmax(digit_predict[a]),np.argmax(digits_train[a]))

In [ ]:
digits_test.iloc[0]

In [ ]:
#heatmap from confusion matrix
import seaborn as sns
df = pd.DataFrame(B)
sns.heatmap(df, annot=True,fmt = "d" ,cmap='Blues')